# **BGM CGM Alignment**

This program compares fingerstick (BGM) readings with the nearest continuous glucose monitor (CGM) values within a 15-minute window. It calculates the percent difference between the two to assess how closely they align. The results are saved to CSV for visualization in Tableau.

Run on Python 3.13 | No errors | No warnings

In [1]:
# Import packages

# For data manipulation
import numpy as np
import pandas as pd

# for displaying and modifying the working directory
import os as os

# For working with datetime objects
from datetime import datetime

In [2]:
# Load BGM dataset
os.chdir(r'D:\OneDrive\Documents\Python\Current\Blood Glucose Readings\BGM') # absolute path, using \ and r prefix
df0 = pd.read_csv("Blood Glucose Readings for Analysis.csv", usecols=['Date', 'Time', 'DateTime', 'Value'])

In [3]:
# Load CGM dataset
os.chdir(r'D:\OneDrive\Documents\Python\Current\Blood Glucose Readings\CGM') # absolute path, using \ and r prefix
df1 = pd.read_csv("Clarity Readings for Analysis.csv", usecols=['Date', 'Time', 'DateTime', 'Value'])

In [4]:
# Set the working directory
os.chdir(r'D:\OneDrive\Documents\Python\Current\Blood Glucose Readings\BGM CGM Range') # absolute path, using \ and r prefix

In [5]:
# Convert the 'DateTime' columns to datetime format
df0['DateTime'] = pd.to_datetime(df0['DateTime'])
df1['DateTime'] = pd.to_datetime(df1['DateTime'])

# Define a function to find CGM readings within 15 minutes before and after each BGM reading
def find_cgm_within_time_window(bgm_time, df1, time_window_minutes=15):
    # Define the time window
    start_time = bgm_time - pd.Timedelta(minutes=time_window_minutes)
    end_time = bgm_time + pd.Timedelta(minutes=time_window_minutes)
    
    # Filter CGM readings that fall within the time window
    filtered_cgm = df1[(df1['DateTime'] >= start_time) & (df1['DateTime'] <= end_time)]
    return filtered_cgm

# Apply this function to each BGM reading and store the results
bgm_cgm_matches = []

for _, row in df0.iterrows():
    matching_cgm = find_cgm_within_time_window(row['DateTime'], df1)
    if not matching_cgm.empty:
        # For each BGM reading, store the matched CGM readings
        for _, cgm_row in matching_cgm.iterrows():
            bgm_cgm_matches.append({
                'BGM DateTime': row['DateTime'],
                'BGM Value': row['Value'],
                'CGM DateTime': cgm_row['DateTime'],
                'CGM Value': cgm_row['Value']
            })

# Convert the results into a DataFrame
df2 = pd.DataFrame(bgm_cgm_matches)

In [6]:
# Display the first 5 rows of the combined DataFrame
df2.head()

,BGM DateTime,BGM Value,CGM DateTime,CGM Value
0,2025-05-16 06:13:00,95,2025-05-16 06:01:57,108
1,2025-05-16 06:13:00,95,2025-05-16 06:06:57,106
2,2025-05-16 06:13:00,95,2025-05-16 06:11:58,107
3,2025-05-16 06:13:00,95,2025-05-16 06:16:57,109
4,2025-05-16 06:13:00,95,2025-05-16 06:21:58,110


In [7]:
# Display basic information about the combined DataFrame
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1098 entries, 0 to 1097
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   BGM DateTime  1098 non-null   datetime64[ns]
 1   BGM Value     1098 non-null   int64         
 2   CGM DateTime  1098 non-null   datetime64[ns]
 3   CGM Value     1098 non-null   int64         
dtypes: datetime64[ns](2), int64(2)
memory usage: 34.4 KB


In [8]:
# Remove the seconds from the DateTime columns
df2['BGM DateTime'] = df2['BGM DateTime'].dt.floor('min')
df2['CGM DateTime'] = df2['CGM DateTime'].dt.floor('min')

In [9]:
# Delete rows prior to 9/1/2024
df2 = df2[df2['BGM DateTime'] > '2024-08-30']

In [10]:
# Add a Date column to df2 that is the date of the BGM DateTime
df2['Date'] = df2['BGM DateTime'].dt.date

In [11]:
# Add a column for difference
df2['Difference'] = df2['CGM Value'] - df2['BGM Value']

In [12]:
# Add a column for the absolute value of the difference
df2['Abs Difference'] = df2['Difference'].abs()

In [13]:
# Verify the changes
df2.head()

,BGM DateTime,BGM Value,CGM DateTime,CGM Value,Date,Difference,Abs Difference
0,2025-05-16 06:13:00,95,2025-05-16 06:01:00,108,2025-05-16,13,13
1,2025-05-16 06:13:00,95,2025-05-16 06:06:00,106,2025-05-16,11,11
2,2025-05-16 06:13:00,95,2025-05-16 06:11:00,107,2025-05-16,12,12
3,2025-05-16 06:13:00,95,2025-05-16 06:16:00,109,2025-05-16,14,14
4,2025-05-16 06:13:00,95,2025-05-16 06:21:00,110,2025-05-16,15,15


In [14]:
# Find the minimum Abs Difference for each BGM DateTime
df3 = df2.loc[df2.groupby('BGM DateTime')['Abs Difference'].idxmin()]

In [15]:
# Range is Difference / BGM Value
df3['Range'] = df3['Difference'] / df3['BGM Value']
# Round the Range to 2 decimal places
df3['Range'] = df3['Range'].round(2)

In [16]:
# Calculate the Range Within using a min value of 0.10
range_abs = np.maximum(np.ceil(np.abs(df3['Range']) * 10) / 10, 0.10)
df3['Range Within'] = np.where(df3['Range'] == 0, 0.10, np.sign(df3['Range']) * range_abs)

In [17]:
# Load location rules
df4 = pd.read_excel('CGM Locations.xlsx', sheet_name='Locations')

In [18]:
# Convert 'Date' column to datetime
df4['Date'] = pd.to_datetime(df4['Date'])

In [19]:
# Convert Date and Time columns in df4 to a single datetime column
df4['DateTime'] = pd.to_datetime(df4['Date'].astype(str) + " " + df4['Time'].astype(str))

In [20]:
# Sort both DataFrames by DateTime
df4 = df4.sort_values('DateTime')
df3 = df3.sort_values('BGM DateTime')

In [21]:
# Perform an asof merge to assign the most recent CGM location
df5 = pd.merge_asof(
    df3,
    df4[['DateTime', 'Location']],
    left_on='BGM DateTime',
    right_on='DateTime',
    direction='backward'
)

In [23]:
# Show the last few rows of df5 to inspect the most recent data
df5.tail(10)

,BGM DateTime,BGM Value,CGM DateTime,CGM Value,Date,Difference,Abs Difference,Range,Range Within,DateTime,Location
173,2025-08-09 07:15:00,90,2025-08-09 07:04:00,91,2025-08-09,1,1,0.01,0.1,2025-08-03 13:30:00,Left - Middle Third
174,2025-08-09 19:27:00,84,2025-08-09 19:39:00,87,2025-08-09,3,3,0.04,0.1,2025-08-03 13:30:00,Left - Middle Third
175,2025-08-10 08:22:00,93,2025-08-10 08:19:00,95,2025-08-10,2,2,0.02,0.1,2025-08-03 13:30:00,Left - Middle Third
176,2025-08-10 19:04:00,125,2025-08-10 19:09:00,135,2025-08-10,10,10,0.08,0.1,2025-08-03 13:30:00,Left - Middle Third
177,2025-08-11 07:16:00,91,2025-08-11 07:14:00,91,2025-08-11,0,0,0.00,0.1,2025-08-03 13:30:00,Left - Middle Third
178,2025-08-11 19:06:00,119,2025-08-11 19:09:00,122,2025-08-11,3,3,0.03,0.1,2025-08-03 13:30:00,Left - Middle Third
179,2025-08-12 07:17:00,89,2025-08-12 07:04:00,91,2025-08-12,2,2,0.02,0.1,2025-08-03 13:30:00,Left - Middle Third
180,2025-08-12 19:04:00,146,2025-08-12 19:09:00,145,2025-08-12,-1,1,-0.01,-0.1,2025-08-03 13:30:00,Left - Middle Third
181,2025-08-13 07:24:00,89,2025-08-13 07:09:00,89,2025-08-13,0,0,0.00,0.1,2025-08-03 13:30:00,Left - Middle Third
182,2025-08-13 19:06:00,119,2025-08-13 19:09:00,126,2025-08-13,7,7,0.06,0.1,2025-08-03 13:30:00,Left - Middle Third


In [24]:
# Create a column for AM/PM based on the BGM DateTime
df5['AM/PM'] = df5['BGM DateTime'].dt.strftime('%p')    

In [25]:
# Order the columns in df5
df5 = df5[['Date', 'AM/PM', 'BGM DateTime', 'BGM Value', 'CGM DateTime', 'CGM Value', 'Difference', 'Abs Difference', 'Range', 'Range Within', 'Location']]

In [26]:
# Replace NaN values in 'Location' with 'Unknown'
df5['Location'] = df5['Location'].fillna('Unknown')

In [27]:
# Display the first 5 rows of df5
df5.head()

,Date,AM/PM,BGM DateTime,BGM Value,CGM DateTime,CGM Value,Difference,Abs Difference,Range,Range Within,Location
0,2025-05-16,AM,2025-05-16 06:13:00,95,2025-05-16 06:06:00,106,11,11,0.12,0.2,Right - Middle Third
1,2025-05-16,PM,2025-05-16 19:04:00,89,2025-05-16 19:11:00,106,17,17,0.19,0.2,Right - Middle Third
2,2025-05-17,AM,2025-05-17 07:16:00,91,2025-05-17 07:12:00,104,13,13,0.14,0.2,Right - Middle Third
3,2025-05-17,PM,2025-05-17 19:02:00,89,2025-05-17 19:07:00,113,24,24,0.27,0.3,Right - Middle Third
4,2025-05-18,AM,2025-05-18 08:20:00,94,2025-05-18 08:32:00,105,11,11,0.12,0.2,Right - Middle Third


In [28]:
# Display basic information about df5
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            183 non-null    object        
 1   AM/PM           183 non-null    object        
 2   BGM DateTime    183 non-null    datetime64[ns]
 3   BGM Value       183 non-null    int64         
 4   CGM DateTime    183 non-null    datetime64[ns]
 5   CGM Value       183 non-null    int64         
 6   Difference      183 non-null    int64         
 7   Abs Difference  183 non-null    int64         
 8   Range           183 non-null    float64       
 9   Range Within    183 non-null    float64       
 10  Location        183 non-null    object        
dtypes: datetime64[ns](2), float64(2), int64(4), object(3)
memory usage: 15.9+ KB


In [29]:
# Save df5 to CSV
df5.to_csv('BGM CGM Range.csv', index=False)